This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [ ]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [8]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=%s&start_date=2017-01-02&end_date=2017-01-03' % API_KEY)
print(r.json())

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2017-01-02', 'end_date': '2017-01-03', 'frequency': 'daily', 'data': [['2017-01-03', 35.9, 35.93, 35.34, 35.48, None, 70618.0, 2515473.0, None, None, None], ['2017-01-02', 34.99, 35.94, 34.99, 35.8, None, 44700.0, 1590561.0, None, None, None]], 'collapse': None, 'order': None}}


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [9]:
#1
AFX_data_2017 = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=%s&?start_date=2017-01-01&end_date=2017-12-31' % API_KEY)
#2
response = AFX_data_2017.json()

lowest_opening_price = highest_opening_price = largest_daily_change = 0
daily_volume = []
daily_close = []

for day in response['dataset_data']['data']:
    try:
        date, open_price, high, low, close, change, volume, *rest = day
        daily_volume.append(volume)
        if not open_price:
            open_price = daily_close[-1]
        if (close):
            daily_close.append(close)
            daily_change = abs(close - open_price)
        lowest_opening_price = open_price if not lowest_opening_price or open_price < lowest_opening_price else lowest_opening_price
        highest_opening_price = open_price if open_price > highest_opening_price else highest_opening_price
        largest_daily_change = daily_change if daily_change > largest_daily_change else largest_daily_change
    except:
        print(day)


def mean(values):
    return sum(values) / len(values)


def median(values):
    sorted_values = sorted(values)
    if len(sorted_values) == 2:
        return sorted_values[1]
    mid_point = len(values) // 2
    return mean([sorted_values[mid_point], sorted_values[mid_point + 1]]) if len(values) % 2 == 0 else sorted_values[mid_point]


# 3
print('Lowest opening price: %f' % lowest_opening_price)
print('Highest opening price: %f' % highest_opening_price)

# 4
print('Largest change in any one day: %f' % largest_daily_change)

# 5
largest_change_between_days = max(daily_close) - min(daily_close)
print('Largest change between any two days: %f' % largest_change_between_days)

# 6
print('Average daily volume: %f' % mean(daily_volume))

# 7
print('Median trading volume: %f' % median(daily_volume))

Lowest opening price: 4.100000
Highest opening price: 53.110000
Largest change in any one day: 7.000000
Largest change between any two days: 49.020000
Average daily volume: 47040.761726
Median trading volume: 34960.000000
